In [1]:
from sklearn.model_selection import train_test_split
from pathlib import Path
import matplotlib.image as img
import json
import numpy as np
from tqdm import tqdm
import h5py

In [2]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [3]:
# Importing Data
# Notes:

# Orientation
# 1: portrait
# 2: portrait, upside down (iPad only)
# 3: landscape, with home button on the right
# 4: landscape, with home button on the left

In [4]:
# Helper Functions from EDA
def ejson(p, fn): # extract json
    with open((p/fn).as_posix()) as f: return json.load(f) 
## Simple Json Reading Functions
def info_data(p): return ejson(p, 'info.json')
def dot_data(p): return ejson(p, 'dotInfo.json')
def frame_data(p): return ejson(p, 'frames.json')    
def screen_data(p): return ejson(p, 'screen.json')
def get_frame(p, img_fn): return img.imread(p/'..'/'..'/'gazecapture-224x224'/p.name/'frames'/img_fn)
## Larger Helper Functions
def coordinate_data(p): 
    data = dot_data(p)
    return data['XCam'], data['YCam'] # we want relative to camera coords

In [5]:
# dset_path = Path('ml/gazecapture/')
dset_path = Path('../gazecapture/')

In [6]:
from multiprocessing import Pool
import time
from concurrent.futures import ThreadPoolExecutor
import functools

In [ ]:
# Extracting and storing X values
def extract_photo(f, case): return get_frame(case, f)

def extract(case):
    F = frame_data(case) # list of file names correlated to camera images 
    with Pool(6) as p:  # multi core usale
        x = [p.map(functools.partial(extract_photo, case=case), F)]
    return x
 
f = h5py.File('X-0.hdf5', 'w')
bs = 500 # bs dsets per hp5y file
for i, case in enumerate(list(dset_path.iterdir())):
    cn = case.name
    X = extract(case)[0]
    dset = f.create_dataset('dset'+cn, (len(X), 224, 224, 3))
    dset[...] = X
    
    if i % bs == 0 and i!= 0:
        f.close()
        f = h5py.File('X-{}.hdf5'.format(i//bs), 'w')

In [ ]:
f = h5py.File('X-0.hdf5', 'r')
for k in list(f.keys()):
    print(f[k].shape)

In [7]:
# # Y_Xpoint, Y_Ypoint = [], []

# def extract_photo(f, case): return get_frame(case, f)

# def extract(case):
# #     XCam, YCam = coordinate_data(case) # list (x, y) coordinates rel to camera 
#     F = frame_data(case) # list of file names correlated to camera images 
# #     O = screen_data(case)['Orientation']
# #     dset_class = info_data(case)['Dataset'] # used for splitting into test / train / val
    
#     with Pool(6) as p:
#         x = [p.map(functools.partial(extract_photo, case=case), F)]
#     return x
    
# #     for i in range(len(F)): # iterate each x, y value
# #         x = get_frame(case, F[i]) # get the photo as matrix
# #         xpoint, ypoint = XCam[i], YCam[i] # coordinate to predict 
        
# #         X.append(x)
# #         Y_Xpoint.append(xpoint)
# #         Y_Ypoint.append(ypoint)
# #         o = O[i] 

# X = np.zeros((31277, 224, 224, 3))
# for i, case in enumerate(list(dset_path.iterdir())[:20]):
#     X[i] = X + extract(case)

# # with Pool(6) as p:
# #     X = [p.map(extract, list(dset_path.iterdir())[:20])]

# # X = np.asarray(X)
# # Y_Xpoint, Y_Ypoint = np.asarray(Y_Xpoint), np.asarray(Y_Ypoint)

In [30]:
# training and validation split
x_train, x_val, y_xpoint_train, y_xpoint_val, y_ypoint_train, y_ypoint_val \
    = train_test_split(X, Y_Xpoint, Y_Ypoint, test_size=0.2, random_state=1)

In [33]:
x_train.shape, y_xpoint_train.shape, y_ypoint_train.shape

((81, 640, 480, 3), (81,), (81,))

In [15]:
N = 0
for i in range(len(X)):
    N += len(X[i]) 
N

31277

In [42]:
bs = 20
for i in range(bs):
    X[0] = np.asarray(list(X[0]) + list(X[1]))
    print(i)

MemoryError: 